# In this exercise we aim at classifying digits using the famous MNIST digits dataset. The dataset consists of 60000 training images and 10000 test images of handwritten digits. Each image has size 28\*28, and has assigned a label from zero to nine, denoting the digits value, therefore we can use this dataset for supervised training. Download the MNIST dataset from http://yann.lecun.com/exdb/mnist/. You can download an import script for python from http://g.sweyla.com/blog/2012/mnist-numpy/. Import the dataset and, in order to get a vectorial representation, flatten the input images, such that each image is represented by a 784-dimensional vector. You should also flatten the labels to get a vectorial representation from a one-dimensional matrix, as these representations are not equivalent in theano.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import os, struct
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros

mnist_path="/path/to/mnist"

def load_mnist(dataset="training", digits=np.arange(10), path="."):
    """
    Loads MNIST files into 3D numpy arrays

    Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
    """

    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [33]:
import theano
#Load the images for training and testing
images, label = load_mnist(dataset="training" , digits=np.arange(10), path=mnist_path)
test_images, test_label = load_mnist(dataset="testing" , digits=np.arange(10), path=mnist_path)
print label.shape
#Flatten the images to get a vectorial representation
images = np.asarray([i.flatten() for i in images])
test_images = np.asarray([i.flatten() for i in test_images]).astype(theano.config.floatX)
#get the maximum for normalization and normalize to values in range [0,1]
maxval = np.max(images)
images = (images * (1.0/maxval)).astype(theano.config.floatX)
test_images = (test_images * (1.0/maxval))
#flatten the labels to get a vectorial representation
label = label.flatten()
test_label = test_label.flatten()
print test_images.shape
print images.shape
print label.shape
print test_label.shape

(60000, 1)
(10000, 784)
(60000, 784)
(60000,)
(10000,)


Given the data, we want to define an MLP for classification. Construct the following network:
* Input $x$: 768-dimensional (i.e. 768 visible units representing the flattened 28\*28 pixel images)
* 100 hidden units $h$
* 10 output units $y$ representing the label, with a value close to one in the $i$-th class representing a high probability of the input representing the digit $i$
* Use a batch size of 100 and a learning rate of 2
* Train 10 epochs. An epoch corresponds to a training interval where each training image is considered once.
* Initialize all input weights of the hidden layer with a random uniform distribution in the range [-0.007,0.007]
* Initialize all input weights of the top classification layer with a random uniform distribution in the range [-0.05,0.05]
* Initialize all biases with zero
* Use a sigmoid activation function for the first layer and softmax for the second layer
* Optimize with negative log-likelihood as a cost function.

The task should be possible to achieve with the knowledge gained from the exercises last week. If you need additional examples you can borrow some code from the deep learning tutorials on http://deeplearning.net/tutorial/. We recommend however that you construct a minimal version of the network on your own without any class structure to gain better insights into the working of Theano. After defining the necessary variables, the network structure can be defined in three lines of code. Defining the updates and the training function can be written in 8 lines of code, and training can be achieved in an additinal five lines.

Computing the loss in this special case where labels are not given as a non-zero entry in a zero-valued vector is not trivial. If ${\tt hiddenOut}$ is the output of the hidden layer, then the loss can be computed as ${\tt loss=-T.mean(T.log(hiddenOut)[T.arange(y.shape[0]), y])}$. The mean of this expression simply takes the mean over a minibatch of 100 training examples. $\tt T.log(hiddenOut)$ takes the logarithm of the outputs of the hidden layer, and $[T.arange(y.shape[0]), y]$ picks the correct entries from the output matrix of size ${\tt (minibatch, \#labels)}$

In [41]:
import theano
import theano.tensor as T
import numpy as np

learning_rate = T.scalar('lr')
index = T.lscalar('index')
batch_size = 100
train_set_x = theano.shared(value=images, name='train_set_x')
train_set_y = T.cast(theano.shared(value=label, name='train_set_y'), 'int32')
visible_units=images.shape[1]
hidden_units=100
output_units=10
rng = numpy.random.RandomState(1242)
epochs = 10
n_train_batches = images.shape[0]/batch_size
learn_rate = 2

########################################
##########INPUTS AND WEIGHTS############
########################################
x = T.matrix('x')
#output
y = T.ivector('y')
#weights first layer
W1 = theano.shared(value=numpy.asarray(rng.uniform(low=-0.007,high=0.007,size=(visible_units,hidden_units)),dtype=theano.config.floatX), name='W1')
#bias first layer
b1 = theano.shared(value=numpy.zeros((hidden_units,),dtype=theano.config.floatX), name='b1')
#weights second layer
W2 = theano.shared(value=numpy.asarray(rng.uniform(low=-0.05,high=0.05,size=(hidden_units,output_units)),dtype=theano.config.floatX), name='W2')
#bias second layer
b2 = theano.shared(value=numpy.zeros((output_units,),dtype=theano.config.floatX), name='b2')

########################################
##############  NETWORK  ###############
########################################
layer1out = T.nnet.sigmoid(T.dot(x, W1)+b1)
layer2out = T.nnet.softmax(T.dot(layer1out, W2)+b2)
loss = -T.mean(T.log(layer2out)[T.arange(y.shape[0]), y])

prediction = T.argmax(layer2out)

########################################
#######  GRADIENT AND UPDATES  #########
########################################
params = [W1,b1,W2,b2]
gradient = T.grad(loss, params)
updates = []
for p, g in zip(params, gradient):
   updates.append((p, p - learning_rate * g))

In [42]:
train = theano.function([index, learning_rate], loss, updates=updates,
     givens = {x: train_set_x[(index * batch_size): ((index + 1) * batch_size)],
               y: train_set_y[(index * batch_size): ((index + 1) * batch_size)]})

In [43]:
for epoch in xrange(epochs):
    # go through training set
    c = []
    for batch_index in xrange(n_train_batches):
        c.append(train(batch_index, learn_rate))
    print 'Refinement epoch %d, cost ' % epoch, numpy.mean(c)

Refinement epoch 0, cost  0.444167
Refinement epoch 1, cost  0.168001
Refinement epoch 2, cost  0.124324
Refinement epoch 3, cost  0.100445
Refinement epoch 4, cost  0.0843293
Refinement epoch 5, cost  0.0723783
Refinement epoch 6, cost  0.0629108
Refinement epoch 7, cost  0.0551432
Refinement epoch 8, cost  0.0485799
Refinement epoch 9, cost  0.0429254


The actual output class of the network can be easily derived by taking the ${\tt argmax}_i y_i$. Evaluate your classifier on the test set and calculate your error rate. 

In [44]:
data= T.matrix(name='data')
test = theano.function([data], prediction,
     givens = {x: data})

evaluated = 0.0
correct = 0.0
for i in range(0,test_images.shape[0]):
    out = test(test_images[i].reshape(1,test_images[i].shape[0]))
    if out == test_label[i]:
        correct = correct+1
    evaluated = evaluated +1

print "Correctly classified: " + str(correct/evaluated*100)

Correctly classified: 97.2
